In [ ]:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")

In [20]:
#counts location
infile = "./MBsimple_counts_fixed.txt"
library(DESeq2)


Loading required package: S4Vectors

Loading required package: stats4



Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: GenomicRanges

Loading required package: GenomeInfoDb

Loading required package: SummarizedExperiment

Loading required package: 

In [21]:
print(infile)

[1] "./MBsimple_counts_fixed.txt"


In [22]:
# Extract the experimental design from the command line.
#design for 3 reps 4 conditions
design = c( "3", "3", "3", "3")
design

[1] "3" "3" "3" "3"

In [23]:
# Find the desing counts.
reps1 = as.integer(design[1])
reps2 = as.integer(design[2])
reps3 = as.integer(design[3])
reps4 = as.integer(design[4])

In [24]:
# Set up the conditions based on the experimental setup.
MB468_000 = rep("MB468_000", reps1)
MB468_30 = rep("MB468_30", reps2)
MB468_120 = rep("MB468_120", reps3)
MB468_720 = rep("MB468_720", reps4)

In [25]:
MB468_000

[1] "MB468_000" "MB468_000" "MB468_000"

In [26]:
# Read the data from the standard input.
# path <- "./MBsimple_counts_fixed.txt"
path <- "./Counts/MBsimple_counts.txt"
counts = read.table(path, header=TRUE, sep="\t", row.names=1 )
head(counts)

,MB468_000_01,MB468_000_02,MB468_000_03,MB468_030_01,MB468_030_02,MB468_030_03,MB468_120_01,MB468_120_02,MB468_120_03,MB468_720_01,MB468_720_02,MB468_720_03
,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
C1orf141,0,0,0,0,0,0,0,0,0,0,0,0
PKP1,48,42,28,20,11,10,234,170,272,646,398,562
RERE,3487,3719,2234,4534,2760,1853,2525,1645,2435,4331,2484,4481
CSMD2,0,1,2,0,0,0,0,0,0,0,0,0
HIVEP3,598,638,350,533,362,208,447,241,446,1082,523,1011
TTLL7,222,250,161,356,252,115,117,116,180,218,127,202


In [27]:
# Some tools generate the estimated counts as real numbers
# DESeq 2 allows only integers. We need to convert real numbers to rounded integers.
numeric_idx = sapply(counts, mode) == 'numeric'
counts[numeric_idx] = round(counts[numeric_idx], 0)
head(counts)

,MB468_000_01,MB468_000_02,MB468_000_03,MB468_030_01,MB468_030_02,MB468_030_03,MB468_120_01,MB468_120_02,MB468_120_03,MB468_720_01,MB468_720_02,MB468_720_03
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C1orf141,0,0,0,0,0,0,0,0,0,0,0,0
PKP1,48,42,28,20,11,10,234,170,272,646,398,562
RERE,3487,3719,2234,4534,2760,1853,2525,1645,2435,4331,2484,4481
CSMD2,0,1,2,0,0,0,0,0,0,0,0,0
HIVEP3,598,638,350,533,362,208,447,241,446,1082,523,1011
TTLL7,222,250,161,356,252,115,117,116,180,218,127,202


In [28]:
# Build the dataset
samples = names(counts)
samples

[1] "MB468_000_01" "MB468_000_02" "MB468_000_03" "MB468_030_01" "MB468_030_02"
 [6] "MB468_030_03" "MB468_120_01" "MB468_120_02" "MB468_120_03" "MB468_720_01"
[11] "MB468_720_02" "MB468_720_03"

In [29]:
condition = factor(c(MB468_000, MB468_30, MB468_120, MB468_720))
condition

[1] MB468_000 MB468_000 MB468_000 MB468_30  MB468_30  MB468_30  MB468_120
 [8] MB468_120 MB468_120 MB468_720 MB468_720 MB468_720
Levels: MB468_000 MB468_120 MB468_30 MB468_720

In [30]:
colData = data.frame(samples=samples, condition=condition)
colData

samples,condition
<chr>,<fct>
MB468_000_01,MB468_000
MB468_000_02,MB468_000
MB468_000_03,MB468_000
MB468_030_01,MB468_30
MB468_030_02,MB468_30
MB468_030_03,MB468_30
MB468_120_01,MB468_120
MB468_120_02,MB468_120
MB468_120_03,MB468_120


In [31]:
# Create DESEq2 dataset.
dds = DESeqDataSetFromMatrix(countData=counts, colData=colData, design = ~condition)

converting counts to integer mode



In [32]:
# Run deseq2.
dds = DESeq(dds)

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing



In [33]:
# Extract the DESeq2 results.
# res = results(dds)
res1 = results(dds, contrast = c("condition", "MB468_30", "MB468_000"))
res2 = results(dds, contrast = c("condition", "MB468_120", "MB468_000"))
res3 = results(dds, contrast = c("condition", "MB468_720", "MB468_000"))

In [34]:
# Turn the DESeq2 results into data frames.
# data = data.frame(res)
data1 = data.frame(res1)
data2 = data.frame(res2)
data3 = data.frame(res3)

In [35]:
# Rename columns for what they are.
# names(data)[names(data)=="pvalue"] <-"PValue"
# names(data)[names(data)=="padj"] <-"FDR"
names(data1)[names(data1)=="pvalue"] <-"PValue"
names(data1)[names(data1)=="padj"] <-"FDR"

names(data2)[names(data2)=="pvalue"] <-"PValue"
names(data2)[names(data2)=="padj"] <-"FDR"

names(data3)[names(data3)=="pvalue"] <-"PValue"
names(data3)[names(data3)=="padj"] <-"FDR"

head(data1)

,baseMean,log2FoldChange,lfcSE,stat,PValue,FDR
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C1orf141,0.0000000,NA,NA,NA,NA,NA
PKP1,180.5116956,-1.3777156,0.28498192,-4.8343965,1.335502e-06,1.122907e-05
RERE,2872.0008215,0.1036148,0.05634364,1.8389788,6.591831e-02,1.619187e-01
CSMD2,0.2815645,-2.4720352,3.95762771,-0.6246255,5.322169e-01,NA
HIVEP3,483.1932856,-0.3714033,0.09730493,-3.8169011,1.351383e-04,7.927503e-04
TTLL7,185.8168583,0.3055194,0.14298989,2.1366506,3.262642e-02,9.255495e-02


In [36]:
data1$foldChange = 2 ^ data1$log2FoldChange
data1$PAdj = p.adjust(data1$PValue, method="hochberg")


data2$foldChange = 2 ^ data2$log2FoldChange
data2$PAdj = p.adjust(data2$PValue, method="hochberg")


data3$foldChange = 2 ^ data3$log2FoldChange
data3$PAdj = p.adjust(data3$PValue, method="hochberg")
head(data1)

,baseMean,log2FoldChange,lfcSE,stat,PValue,FDR,foldChange,PAdj
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C1orf141,0.0000000,NA,NA,NA,NA,NA,NA,NA
PKP1,180.5116956,-1.3777156,0.28498192,-4.8343965,1.335502e-06,1.122907e-05,0.3848277,0.02517821
RERE,2872.0008215,0.1036148,0.05634364,1.8389788,6.591831e-02,1.619187e-01,1.0744622,1.00000000
CSMD2,0.2815645,-2.4720352,3.95762771,-0.6246255,5.322169e-01,NA,0.1802367,1.00000000
HIVEP3,483.1932856,-0.3714033,0.09730493,-3.8169011,1.351383e-04,7.927503e-04,0.7730302,1.00000000
TTLL7,185.8168583,0.3055194,0.14298989,2.1366506,3.262642e-02,9.255495e-02,1.2358635,1.00000000


In [37]:
# Compute the false discovery counts on the sorted table.
#data$falsePos = 1:nrow(data) * data$FDR
data1$falsePos = 1:nrow(data1) * data1$FDR

data2$falsePos = 1:nrow(data2) * data2$FDR

data3$falsePos = 1:nrow(data3) * data3$FDR
head(data1)

,baseMean,log2FoldChange,lfcSE,stat,PValue,FDR,foldChange,PAdj,falsePos
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C1orf141,0.0000000,NA,NA,NA,NA,NA,NA,NA,NA
PKP1,180.5116956,-1.3777156,0.28498192,-4.8343965,1.335502e-06,1.122907e-05,0.3848277,0.02517821,2.245814e-05
RERE,2872.0008215,0.1036148,0.05634364,1.8389788,6.591831e-02,1.619187e-01,1.0744622,1.00000000,4.857562e-01
CSMD2,0.2815645,-2.4720352,3.95762771,-0.6246255,5.322169e-01,NA,0.1802367,1.00000000,NA
HIVEP3,483.1932856,-0.3714033,0.09730493,-3.8169011,1.351383e-04,7.927503e-04,0.7730302,1.00000000,3.963752e-03
TTLL7,185.8168583,0.3055194,0.14298989,2.1366506,3.262642e-02,9.255495e-02,1.2358635,1.00000000,5.553297e-01


In [38]:
# Rearrange the columns into the same order as other methods do.
#data = data[c(1,7,8,9,2,3,4,5, 10, 6, 11)]
data1 = data1[c(1,7,2,3,4,5, 8, 6, 9)]

data2 = data2[c(1,7,2,3,4,5, 8, 6, 9)]

data3 = data3[c(1,7,2,3,4,5, 8, 6, 9)]
head(data1)

,baseMean,foldChange,log2FoldChange,lfcSE,stat,PValue,PAdj,FDR,falsePos
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
C1orf141,0.0000000,NA,NA,NA,NA,NA,NA,NA,NA
PKP1,180.5116956,0.3848277,-1.3777156,0.28498192,-4.8343965,1.335502e-06,0.02517821,1.122907e-05,2.245814e-05
RERE,2872.0008215,1.0744622,0.1036148,0.05634364,1.8389788,6.591831e-02,1.00000000,1.619187e-01,4.857562e-01
CSMD2,0.2815645,0.1802367,-2.4720352,3.95762771,-0.6246255,5.322169e-01,1.00000000,NA,NA
HIVEP3,483.1932856,0.7730302,-0.3714033,0.09730493,-3.8169011,1.351383e-04,1.00000000,7.927503e-04,3.963752e-03
TTLL7,185.8168583,1.2358635,0.3055194,0.14298989,2.1366506,3.262642e-02,1.00000000,9.255495e-02,5.553297e-01


In [39]:
# Get the normalized counts.
normed = counts(dds, normalized=TRUE)
head(normed)

,MB468_000_01,MB468_000_02,MB468_000_03,MB468_030_01,MB468_030_02,MB468_030_03,MB468_120_01,MB468_120_02,MB468_120_03,MB468_720_01,MB468_720_02,MB468_720_03
C1orf141,0.00000,0.0000000,0.000000,0.0000,0.00000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
PKP1,40.20504,31.1820996,36.908802,13.7887,11.37427,17.81181,292.7700,305.5723,315.5889,367.8294,401.5631,331.5460
RERE,2920.72862,2761.1006802,2944.795139,3125.8994,2853.90801,3300.52877,3159.1635,2956.8612,2825.2165,2466.0513,2506.2378,2643.5189
CSMD2,0.00000,0.7424309,2.636343,0.0000,0.00000,0.00000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
HIVEP3,500.88779,473.6709422,461.360026,367.4690,374.31692,370.48569,559.2658,433.1937,517.4729,616.0858,527.6821,596.4288
TTLL7,185.94831,185.6077360,212.225612,245.4389,260.57421,204.83584,146.3850,208.5081,208.8456,124.1282,128.1370,119.1678


In [40]:
# Write the results to the standard output.
# write.csv(total, file=outfile, row.names=FALSE, quote=FALSE)
write.csv(data1, file="./Counts/ComMB_0_30.csv", row.names=TRUE, quote=FALSE)

write.csv(data2, file="./Counts/ComMB_0_120.csv", row.names=TRUE, quote=FALSE)

write.csv(data3, file="./Counts/ComMB_0_720.csv", row.names=TRUE, quote=FALSE)